In [1]:
cd '/mmfs1/data/aglinska/BC-MRI-AE/Colab Notebooks'

/mmfs1/data/aglinska/BC-MRI-AE/Colab Notebooks


In [2]:
import pickle

import numpy as np
import pandas as pd 

from importlib import reload

import helper_funcs;reload(helper_funcs);from helper_funcs import *
import make_models2;reload(make_models2);from make_models2 import *

from functools import partial
from sklearn.decomposition import PCA

from matplotlib import pyplot as plt

from datetime import datetime

import shutil
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

In [3]:
dataFnOut = '/mmfs1/data/aglinska/BC-MRI-AE/Data/ABIDE-Anat-64iso-S982.npz'
dfFnOut = '/mmfs1/data/aglinska/BC-MRI-AE/Data/ABIDE_legend_S982.csv'

ABIDE_data = np.load(dataFnOut)['data']
df = pd.read_csv(dfFnOut)

patients = df['DxGroup'].values==1
controls = df['DxGroup'].values==2
abide_asd = ABIDE_data[patients,:,:,:]

arr = np.load('../Data/SFARI-Anat-64iso-S121.npz')
dfs = pd.read_csv('../Data/sfari_legend_S121.csv')

SFARI_data = arr['data']
SFARI_subs = arr['subs']

sfari_subs_td = dfs['family_type'].values=='non-familial-control'
sfari_subs_dupl = dfs['family_type'].values=='16p-duplication'
sfari_subs_del = dfs['family_type'].values=='16p-deletion'

In [4]:
patients = df['DxGroup'].values==1
controls = df['DxGroup'].values==2

In [6]:
# LOAD VAE
latent_dim=32;batch_size=16;disentangle=False;gamma=100
encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64, 64, 64, 1), latent_dim=latent_dim, batch_size=batch_size, disentangle=disentangle, gamma=gamma)


encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1), 
                                            latent_dim=32, 
                                            batch_size = batch_size, 
                                            disentangle=True,
                                            gamma=100,
                                            kernel_size = 3,
                                            filters = 48,
                                            intermediate_dim = 128,
                                            nlayers = 2,
                                            bias=True)

#fn = '/mmfs1/data/aglinska/tf_outputs/VAE/Jan12th_16d'
#fn = '/mmfs1/data/aglinska/tf_outputs/VAE/Feb_26_32DB64G1'
#fn = '/mmfs1/data/aglinska/tf_outputs/VAE/Feb_26_32DB64G100_DisTrue'
#fn = '/mmfs1/data/aglinska/tf_outputs/VAE/APR_4th_D32DTrue_G100'
fn = '/mmfs1/data/aglinska/tf_outputs/VAE/20210920-2122'

loss = list()
if os.path.exists(fn+'_loss.pickle'):
    print('loading')
    vae.load_weights(fn)
    loss = pickle.load(open(fn+'_loss.pickle','rb'))
    
print('ready')

ready
loading


In [8]:
%%time 
# LOAD CVAE
latent_dim = 16;batch_size = 32;beta = 1;gamma = 100;disentangle = True
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(latent_dim=latent_dim,beta=beta, disentangle=disentangle, gamma=gamma, bias=True, batch_size = batch_size)
loss = list()    

fdir = '/mmfs1/data/aglinska/tf_outputs/CVAE/'

#fn = f'{now().strftime("%b%d_%H%M")}_D{latent_dim}B{beta}G{gamma}_D{disentangle}'
fn = 'Feb04_0524_D16B1G100_DTrue'
#fn = os.path.join(fdir,'Mar18_1312_CVAE_D16B1G1_adam_rData2')
fn = os.path.join(fdir,'Mar20_2125_CVAE_D16B1G1_ADAM_Bspline_r2_new')

fn = os.path.join(fdir,fn)
fn = '/mmfs1/data/aglinska/tf_outputs/CVAE/Feb23_1551_D16B1G100_DTrue'

loss = pickle.load(open(fn+'_loss.pickle','rb'))
cvae.load_weights(fn)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x1551a8cf5970> and <tensorflow.python.keras.layers.core.TFOpLambda object at 0x1551a8cdf2e0>).
CPU times: user 2.59 s, sys: 1.57 s, total: 4.16 s
Wall time: 1.42 s


In [9]:
n_samples = 10

In [10]:
%%time
salient_vec_abide = np.array([s_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
background_vec_abide = np.array([z_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
vae_vec_abide = np.array([encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])

CPU times: user 33min 25s, sys: 2min 30s, total: 35min 55s
Wall time: 1min 11s


In [11]:
%%time
salient_vec_sfari = np.array([s_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
background_vec_sfari = np.array([z_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
vae_vec_sfari = np.array([encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])

CPU times: user 4min 8s, sys: 18.2 s, total: 4min 26s
Wall time: 9.63 s


In [ ]:
fn = '../Data/latent_vecs100.npz'
np.savez_compressed(fn, 
                    salient_vec_abide=salient_vec_abide,
                    background_vec_abide=background_vec_abide,
                    vae_vec_abide=vae_vec_abide,
                   salient_vec_sfari=salient_vec_sfari,
                   background_vec_sfari=background_vec_sfari,
                   vae_vec_sfari=vae_vec_sfari)

In [ ]:
loaded_data = np.load(fn)
list(loaded_data.keys())

In [ ]:
loaded_data = np.load(fn)
list(loaded_data.keys())